## Using only 2 out of the 12 REST tools

In [33]:
# ONE‑CELL: REST‑wrapped tools ➜ Azure OpenAI function‑calling agent
# -----------------------------------------------------------------
# %pip install --quiet "openai>=1.25.0" requests python-dotenv

import os, json, uuid, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

# 0. config --------------------------------------------------------
load_dotenv()
BASE   = "http://localhost:8000"
MODEL  = os.getenv("AZURE_OPENAI_DEPLOYMENT") or "gpt-4o"

oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_KEY"),
    api_version    = "2025-04-01-preview",
)

# 1. REST wrappers -------------------------------------------------
def rest_search_legislation_section(query: str, size: int = 3):
    r = requests.post(f"{BASE}/legislation/section/search",
                      json={"query": query, "size": size}, timeout=30)
    r.raise_for_status()
    return r.json()

def rest_search_caselaw_section(query: str, size: int = 3):
    r = requests.post(f"{BASE}/caselaw/section/search",
                      json={"query": query, "size": size}, timeout=30)
    r.raise_for_status()
    return r.json()

LOCAL_ROUTER = {
    "search_legislation_section": rest_search_legislation_section,
    "search_caselaw_section":     rest_search_caselaw_section,
}

# 2. tool specs ----------------------------------------------------
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "search_legislation_section",
            "description": "Search UK legislation sections.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "size":  {"type": "integer", "default": 3},
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_caselaw_section",
            "description": "Search court‑judgment sections that cite legislation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"},
                    "size":  {"type": "integer", "default": 3},
                },
                "required": ["query"],
            },
        },
    },
]

# 3. agent loop ----------------------------------------------------
def chat(question: str) -> str:
    msgs = [
        {"role": "system",
         "content": "You are a UK public‑law assistant. Use the tools when helpful."},
        {"role": "user", "content": question},
    ]

    while True:
        resp = oai.chat.completions.create(
            model       = MODEL,
            messages    = msgs,
            tools       = TOOLS,
            tool_choice = "auto",
        ).choices[0].message

        # If no tool call, assistant is giving the final answer
        if resp.tool_calls is None:
            return resp.content

        # Record the assistant message that requested tools
        msgs.append(resp)

        # Execute each requested tool and append its result
        for call in resp.tool_calls:
            fn_name = call.function.name
            args    = json.loads(call.function.arguments)
            py_resp = LOCAL_ROUTER[fn_name](**args)

            msgs.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": fn_name,
                "content": json.dumps(py_resp),
            })
        # Loop again—model now sees the tool responses

# 4. demo ----------------------------------------------------------
print(chat(
    # "Summarise the duty in section 5 of the Environment Act 2021 and cite any cases interpreting it."
    # "What duties does the Climate Change Act 2008 impose on the Secretary of State?"
    "what do we know about Mrs Staveley’s estate? You must quote relevant sections from the documents."

))


The following information regarding Mrs Staveley's estate has been drawn from both legislative texts and the judgment summaries:

---

### Legislative Summaries:
1. **No specific mention in legislative excerpts provided.**
   - The legislative texts retrieved did not explicitly address "Mrs Staveley's estate."

---

### Court Judgment Summaries:
The case record ([2020] UKSC 35) and relevant sections reveal:

1. **Divorce and Pension Transfer**:
   - Mrs. Staveley divorced in 2000. The settlement involved transferring her share from the company pension scheme established during her marriage (with Morayford Ltd) into a "section 32 buyout policy."
   - This policy had a clause that any unutilized funds in the pension upon her death would return to the company, potentially benefitting her ex-husband, which she found unacceptable.

   *“...her share of the company pension scheme be transferred to her...However, given the level of her salary with the company, the pension was over-funded, and

## Using all 12 REST tools

In [28]:
# ONE‑CELL: 12 REST tools, but look‑ups marked “requires exact ID”
# ----------------------------------------------------------------
# %pip install --quiet "openai>=1.25.0" requests python-dotenv

import os, json, uuid, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
BASE  = "http://localhost:8000"
MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT") or "gpt-4o"

oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_KEY"),
    api_version    = "2025-04-01-preview",
)

# -------- endpoint table -------------------------------------------------
ENDPOINTS = {
    # --- searches (safe defaults) ---
    "/legislation/section/search":        ("search_legislation_section","Search UK legislation sections.", ["query"]),
    "/legislation/search":                ("search_legislation_act",   "Search UK legislation acts.", ["query"]),
    "/caselaw/section/search":            ("search_caselaw_section",   "Search caselaw sections.", ["query"]),
    "/caselaw/search":                    ("search_caselaw",           "Search caselaw judgments.", ["query"]),
    "/caselaw/reference/search":          ("search_caselaw_reference", "Find cases that cite a given case or legislation.", ["reference_id","reference_type"]),
    "/explanatory_note/section/search":   ("search_explanatory_note_section","Search explanatory note sections.",["query"]),
    "/amendment/search":                  ("search_amendment","Search amendments by legislation.", ["legislation_id"]),
    "/amendment/section/search":          ("search_amendment_section","Search amendments by provision.", ["provision_id"]),

    # --- look‑ups (need precise IDs) ---
    "/legislation/section/lookup":        ("lookup_legislation_sections","**Lookup** all sections of an Act (requires exact legislation_id).", ["legislation_id"]),
    "/legislation/lookup":                ("lookup_legislation","**Lookup** an Act by type + year + number (requires all three).", ["legislation_type","year","number"]),
    "/legislation/text":                  ("lookup_legislation_full_text","**Lookup** full text of an Act (requires exact legislation_id).", ["legislation_id"]),
    "/explanatory_note/legislation/lookup": ("lookup_explanatory_note","**Lookup** explanatory notes for an Act (requires exact legislation_id).",["legislation_id"]),
}

# -------- dynamic wrappers & tool specs ---------------------------------
LOCAL_ROUTER, TOOLS = {}, []

def make_wrapper(path, required):
    def _call(**body):
        missing = [k for k in required if k not in body]
        if missing:
            return {"error": f"missing {missing}"}
        try:
            r = requests.post(f"{BASE}{path}", json=body, timeout=60)
            r.raise_for_status()
            return r.json()
        except requests.HTTPError as e:
            return {"http_error": r.status_code, "detail": r.text[:300]}
    return _call

for path, (name, desc, req) in ENDPOINTS.items():
    LOCAL_ROUTER[name] = make_wrapper(path, req)

    props = {k: {"type": "string"} for k in req}
    if "size" not in req:
        props["size"] = {"type": "integer", "default": 10}

    TOOLS.append({
        "type": "function",
        "function": {
            "name": name,
            "description": desc,                       # <‑‑ lookup note included
            "parameters": {"type":"object",
                           "properties": props,
                           "required": req},
        },
    })

print(f"Registered {len(TOOLS)} tools (look‑ups flagged as exact‑ID)")

# -------- agent loop -----------------------------------------------------
def chat(prompt: str) -> str:
    msgs = [
        {"role":"system",
         "content":"You are a UK public‑law assistant. "
                   "Use the search tools by default; only call a look‑up tool "
                   "when the user supplies an exact ID."},
        {"role":"user","content":prompt},
    ]
    while True:
        reply = oai.chat.completions.create(
            model=MODEL, messages=msgs, tools=TOOLS, tool_choice="auto"
        ).choices[0].message

        if reply.tool_calls is None:
            return reply.content

        msgs.append(reply)  # assistant with tool_calls

        for call in reply.tool_calls:
            args = json.loads(call.function.arguments)
            tool_resp = LOCAL_ROUTER[call.function.name](**args)
            msgs.append({
                "role":"tool",
                "tool_call_id":call.id,
                "name":call.function.name,
                "content":json.dumps(tool_resp),
            })

# -------- demo -----------------------------------------------------------
print(chat(
    # "what do we know about Mrs Staveley’s estate?"
    "What duties does the Climate Change Act 2008 impose on the Secretary of State?"
))


Registered 12 tools (look‑ups flagged as exact‑ID)
The Climate Change Act 2008 wasn't found in the current search index, but this legislation is fundamental in defining the UK government's duties in addressing climate change. If you would like specific information, please let me know, and I can attempt to locate authoritative resources or describe the framework duties generally associated with it.


## 12 tools with trace

In [30]:
# ONE‑CELL: 12 REST tools, but look‑ups marked “requires exact ID”
# ----------------------------------------------------------------
# %pip install --quiet "openai>=1.25.0" requests python-dotenv

import os, json, uuid, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
BASE  = "http://localhost:8000"
MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT") or "gpt-4o"

oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_KEY"),
    api_version    = "2025-04-01-preview",
)

# -------- endpoint table -------------------------------------------------
ENDPOINTS = {
    # --- searches (safe defaults) ---
    "/legislation/section/search":        ("search_legislation_section","Search UK legislation sections.", ["query"]),
    "/legislation/search":                ("search_legislation_act",   "Search UK legislation acts.", ["query"]),
    "/caselaw/section/search":            ("search_caselaw_section",   "Search caselaw sections.", ["query"]),
    "/caselaw/search":                    ("search_caselaw",           "Search caselaw judgments.", ["query"]),
    "/caselaw/reference/search":          ("search_caselaw_reference", "Find cases that cite a given case or legislation.", ["reference_id","reference_type"]),
    "/explanatory_note/section/search":   ("search_explanatory_note_section","Search explanatory note sections.",["query"]),
    "/amendment/search":                  ("search_amendment","Search amendments by legislation.", ["legislation_id"]),
    "/amendment/section/search":          ("search_amendment_section","Search amendments by provision.", ["provision_id"]),

    # --- look‑ups (need precise IDs) ---
    "/legislation/section/lookup":        ("lookup_legislation_sections","**Lookup** all sections of an Act (requires exact legislation_id).", ["legislation_id"]),
    "/legislation/lookup":                ("lookup_legislation","**Lookup** an Act by type + year + number (requires all three).", ["legislation_type","year","number"]),
    "/legislation/text":                  ("lookup_legislation_full_text","**Lookup** full text of an Act (requires exact legislation_id).", ["legislation_id"]),
    "/explanatory_note/legislation/lookup": ("lookup_explanatory_note","**Lookup** explanatory notes for an Act (requires exact legislation_id).",["legislation_id"]),
}

# -------- dynamic wrappers & tool specs ---------------------------------
LOCAL_ROUTER, TOOLS = {}, []

def make_wrapper(path, required):
    def _call(**body):
        missing = [k for k in required if k not in body]
        if missing:
            return {"error": f"missing {missing}"}
        try:
            r = requests.post(f"{BASE}{path}", json=body, timeout=60)
            r.raise_for_status()
            return r.json()
        except requests.HTTPError as e:
            return {"http_error": r.status_code, "detail": r.text[:300]}
    return _call

for path, (name, desc, req) in ENDPOINTS.items():
    LOCAL_ROUTER[name] = make_wrapper(path, req)

    props = {k: {"type": "string"} for k in req}
    if "size" not in req:
        props["size"] = {"type": "integer", "default": 10}

    TOOLS.append({
        "type": "function",
        "function": {
            "name": name,
            "description": desc,                       # <‑‑ lookup note included
            "parameters": {"type":"object",
                           "properties": props,
                           "required": req},
        },
    })

print(f"Registered {len(TOOLS)} tools (look‑ups flagged as exact‑ID)")

# ── agent loop with TRACE ──────────────────────────────────────────
def chat_trace(prompt: str, max_turns: int = 10) -> str:
    msgs = [
        {"role": "system",
         "content": "You are a UK public‑law assistant. "
                    "Use the search tools by default; only call look‑ups when "
                    "the user supplies an exact ID."},
        {"role": "user", "content": prompt},
    ]

    turn = 0
    while turn < max_turns:
        turn += 1
        resp = oai.chat.completions.create(
            model       = MODEL,
            messages    = msgs,
            tools       = TOOLS,
            tool_choice = "auto",
        ).choices[0].message

        # assistant chose to answer directly
        if resp.tool_calls is None:
            print(f"#↩︎ assistant‑final:\n{resp.content[:500]}...\n")
            return resp.content

        # log the assistant's tool requests
        msgs.append(resp)
        for call in resp.tool_calls:
            fn  = call.function.name
            arg = json.loads(call.function.arguments)
            print(f"#A assistant‑calls: {fn} {arg}")

            tool_resp = LOCAL_ROUTER[fn](**arg)
            print(f"#T tool‑returns: {json.dumps(tool_resp)[:120]}...")

            msgs.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": fn,
                "content": json.dumps(tool_resp),
            })

    # safety
    return "Reached max_turns without final answer."

# -------- demo -----------------------------------------------------------
print(chat_trace(
    # "what do we know about Mrs Staveley’s estate?"
    # "Summarise the duty in section 5 of the Environment Act 2021 and cite any cases interpreting it."
    "What duties does the Climate Change Act 2008 impose on the Secretary of State?"
))


Registered 12 tools (look‑ups flagged as exact‑ID)
#A assistant‑calls: search_legislation_section {'query': 'duties of Secretary of State Climate Change Act 2008', 'size': 10}
#T tool‑returns: [{"created_at": "2025-07-29T12:31:58.461257", "id": "http://www.legislation.gov.uk/id/ukpga/2021/30/section/21", "uri": ...
#A assistant‑calls: search_legislation_act {'query': 'Climate Change Act 2008'}
#T tool‑returns: [{"created_at": "2025-07-29T12:28:44.026193", "id": "http://www.legislation.gov.uk/id/ukpga/2020/22", "uri": "http://www...
#A assistant‑calls: lookup_legislation {'legislation_type': 'ukpga', 'year': '2008', 'number': '27'}
#T tool‑returns: {"http_error": 404, "detail": "{\"detail\":\"Legislation not found: ukpga 2008 No. 27\"}"}...
#A assistant‑calls: search_legislation_act {'query': 'Climate Change Act'}
#T tool‑returns: [{"created_at": "2025-07-29T12:28:44.026193", "id": "http://www.legislation.gov.uk/id/ukpga/2020/22", "uri": "http://www...
#↩︎ assistant‑final:
I was unable 